In [1]:
import os
import numpy as np
import nibabel as nib
import glob
import os
from os.path import join
import nibabel as nib
from skimage.measure import label, regionprops
import numpy as np
import math


In [30]:
def connected_component_analysis(seg, thr=0.5):
    mask = seg.copy()
    mask[mask > thr] = 255
    mask[mask <= thr] = 0

    lbl = label(mask)
    blobs = regionprops(lbl)
    raw_bbox_list = [b.bbox for b in blobs]

    return raw_bbox_list


def scale_bbox_old(bbox, ratio, org_shape):
    x1, x2, x3, y1, y2, y3 = bbox
    hsr = (ratio - 1.0) / 2 # half_step_ratio

    x1_new = x1 * (1.0 - hsr)
    x2_new = x2 * (1.0 - hsr)
    x3_new = x3 * (1.0 - hsr)
    y1_new = y1 * (1.0 + hsr)
    y2_new = y2 * (1.0 + hsr)
    y3_new = y3 * (1.0 + hsr)

    x1_new = x1_new if x1_new >=0 else 0
    x2_new = x2_new if x2_new >=0 else 0
    x3_new = x3_new if x3_new >=0 else 0
    y1_new = y1_new if y1_new < org_shape[0] else org_shape[0]-1
    y2_new = y2_new if y2_new < org_shape[1] else org_shape[1]-1
    y3_new = y3_new if y3_new < org_shape[2] else org_shape[2]-1

    return [int(x1_new), int(x2_new), int(x3_new), int(y1_new) ,int(y2_new), int(y3_new)]

def scale_xy_bbox(bbox, ratio, org_shape):
    x1, x2, x3, y1, y2, y3 = bbox
    
    l1 = (y1 - x1) * (ratio-1.0)
    l2 = (y2 - x2) * (ratio-1.0)
    l3 = (y3 - x3) * (4*ratio-1.0)

    x1_new = x1 - (l1/2)
    x2_new = x2 - (l2/2)
    x3_new = x3 - (l3/2)
    y1_new = y1 + (l1/2)
    y2_new = y2 + (l2/2)
    y3_new = y3 + (l3/2)

    x1_new = x1_new if x1_new >=0 else 0
    x2_new = x2_new if x2_new >=0 else 0
    x3_new = x3_new if x3_new >=0 else 0
    y1_new = y1_new if y1_new < org_shape[0] else org_shape[0]-1
    y2_new = y2_new if y2_new < org_shape[1] else org_shape[1]-1
    y3_new = y3_new if y3_new < org_shape[2] else org_shape[2]-1

    return [int(x1_new), int(x2_new), int(x3_new), int(y1_new) ,int(y2_new), int(y3_new)]
    #return [int(x1_new), int(x2_new), int(x3), int(y1_new) ,int(y2_new), int(y3)]


def filter_by_volume(bboxes, min_volume):
    filtered_bboxes = []
    for bbox in bboxes:
        x1, x2, x3, y1, y2, y3 = bbox
        vol = (y1-x1) * (y2-x2) * (y3-x3)
        if vol > min_volume:
            filtered_bboxes.append(bbox)
    
    return filtered_bboxes

def merge_bboxes(bboxes):

    x1 = min([bbox[0] for bbox in bboxes])
    x2 = min([bbox[1] for bbox in bboxes])
    x3 = min([bbox[2] for bbox in bboxes])
    y1 = max([bbox[3] for bbox in bboxes])
    y2 = max([bbox[4] for bbox in bboxes])
    y3 = max([bbox[5] for bbox in bboxes])

    return [x1, x2, x3, y1, y2, y3]
    

def make_square(bbox, shape):
    x1, x2, x3, y1, y2, y3 = bbox
    height, width, depth = shape[0], shape[1], shape[2]
    box_h = y1 - x1
    box_w = y2 - x2
    box_d = y3 - x3


    assert max(box_h, box_w) <= min(height, width)


    #print('box_h', box_h, 'box_w', box_w)

    if box_h > box_w:
        pad_x = math.floor((box_h - box_w)/2)
        pad_y = math.ceil((box_h - box_w)/2)
        assert (pad_x + pad_y) == (box_h - box_w)
        #print('pad_x', pad_x, 'pad_y', pad_y)
        new_x2 = x2 - pad_x
        new_y2 = y2 + pad_y

        if new_y2 >= width:
            new_y2 = width-1
            new_x2 = width - box_h
        
        if new_x2 < 0:
            new_x2 = 0
            new_y2 = box_h           

        square_box = [x1, new_x2, x3, y1, new_y2, y3]
    elif box_w > box_h:
        pad_x = math.floor((box_w - box_h)/2)
        pad_y = math.ceil((box_w - box_h)/2)
        assert (pad_x + pad_y) == (box_w - box_h)
        #print('pad_x', pad_x, 'pad_y', pad_y)
        new_x1 = x1 - pad_x
        new_y1 = y1 + pad_y

        if new_y1 >= height:
            new_y1 = height-1
            new_x1 = height - box_w
        
        if new_x1 < 0:
            new_x1 = 0
            new_y1 = box_w

        square_box = [new_x1, x2, x3, new_y1, y2, y3]
    else:

        square_box = bbox

    
    # Check validity
    x1, x2, x3, y1, y2, y3 = square_box
    #print('h', y1-x1, 'w', y2-x2)
    assert (y1 - x1) == (y2 - x2)
    assert (x1 >= 0) and (x2 >= 0)
    assert (y1 < height) and (y2 < width)
    
    return square_box


In [34]:
def main():

    center = 'allegheny'

    t2_dir = f'/data/ayc9699/dataset/pancreas_ipmn_ilkin/cyst_segmentation/images/reoriented/{center}_PLS'
    seg_dir = f'/data/ayc9699/dataset/pancreas_ipmn_ilkin/cyst_segmentation/labels/reoriented/{center}_PLS'
    cropped_dir = f'/data/ayc9699/dataset/pancreas_ipmn_ilkin/cyst_segmentation/images/reoriented/cropped/{center}'
    cropped_seg_dir = f'/data/ayc9699/dataset/pancreas_ipmn_ilkin/cyst_segmentation/labels/reoriented/cropped/{center}'

    #if not os.path.exists(roi_dir):
    #    os.makedirs(roi_dir)

    for img_name in os.listdir(seg_dir):
        if img_name.startswith('.'): continue
        # Prepare paths
        img_path = join(t2_dir, img_name)
        seg_path = join(seg_dir, img_name)
        outpath = join(cropped_dir, img_name)
        
        # Read niftii image and segmentation
        img = nib.load(img_path)
        seg = nib.load(seg_path)

        # Find Bboxes
        bboxes = connected_component_analysis(seg.get_fdata(), thr=0.5)

        # Filter out small bboxes
        bboxes = filter_by_volume(bboxes, min_volume=5) #25

        # Check if there is any bbox found
        if len(bboxes) == 0:
            print(img_name, 'BBOX is NOT FOUND')
            continue
        else:
            print(img_name, len(bboxes))

        # Merge bboxes
        bbox = merge_bboxes(bboxes)

        # Make it square box
        #print(bbox, img.shape)
        bbox = make_square(bbox, img.shape)
        #print(bbox, img.shape)

        # Enlarge bbox
        x1, x2, x3, y1, y2, y3 = scale_xy_bbox(bbox, ratio=1.2, org_shape=seg.shape)
        #print(x1, x2, x3, y1, y2, y3)
        
        # Bbox to segmentation for visualization
        bbox_mask = np.zeros(seg.shape, dtype=seg.get_data_dtype())
        bbox_mask[x1:y1, x2:y2, x3:y3] = 1
        #bbox_nifti = nib.Nifti1Image(bbox_mask, seg.affine)
        #nib.save(bbox_nifti, join(roi_dir, pid))
        
        # Crop from the source volume
        cropped_img = img.get_fdata()[x1:y1, x2:y2, x3:y3]
        cropped_img_nifti = nib.Nifti1Image(cropped_img, img.affine)
        nib.save(cropped_img_nifti, join(cropped_dir, img_name))

        # Crop from the segmentation
        cropped_seg = seg.get_fdata()[x1:y1, x2:y2, x3:y3]
        cropped_seg_nifti = nib.Nifti1Image(cropped_seg, seg.affine)
        nib.save(cropped_seg_nifti, join(cropped_seg_dir, img_name))

        #print(cropped_img.shape, cropped_seg.shape)
        if cropped_img.shape[2]<16:
            print("FUCKKKKK")


#TOOK OUT NYU 150!!!!!

if __name__ == '__main__':
    main()

ahn_21.nii.gz 2
ahn_29.nii.gz 1
ahn_54.nii.gz 1
ahn_02.nii.gz 1
ahn_23.nii.gz 2
ahn_80.nii.gz 1
ahn_51.nii.gz 4
ahn_09.nii.gz 1
ahn_77.nii.gz 1
ahn_11.nii.gz 1
ahn_48.nii.gz 2
ahn_27.nii.gz 1
ahn_28.nii.gz 1
ahn_35.nii.gz 2
ahn_16.nii.gz 2
ahn_12.nii.gz 1
ahn_05.nii.gz 1
ahn_19.nii.gz 1
ahn_41.nii.gz 1
ahn_58.nii.gz 1
ahn_61.nii.gz 1


In [38]:
import os
import nibabel as nib
import numpy as np
import re

def remove_leading_zeros(number_str):
    return str(int(number_str))  # Convert to int and then back to str to remove leading zeros

def convert_nii_to_numpy(src_dir, target_dir):
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    # Get subdirectories (folders) under the source directory
    subdirs = [subdir for subdir in os.listdir(src_dir) if os.path.isdir(os.path.join(src_dir, subdir))]

    for subdir in subdirs:
        subdir_path = os.path.join(src_dir, subdir)
        for filename in os.listdir(subdir_path):
            if filename.endswith('.nii.gz'):
                nii_file_path = os.path.join(subdir_path, filename)

                # Load the NIfTI file
                img = nib.load(nii_file_path)
                data = img.get_fdata()

                # Extract center name and ID from the filename using regular expression
                match = re.match(r'^([A-Za-z]{3})_([0-9]+)\.nii\.gz$', filename)
                if match:
                    center_name, id_with_zeros = match.groups()
                    id_without_zeros = remove_leading_zeros(id_with_zeros)

                    # Create a new filename with the center name and the ID without leading zeros
                    new_filename = f"{center_name}_{id_without_zeros}.npy"

                    # Create a numpy file path in the target directory with the new filename
                    npy_file_path = os.path.join(target_dir, new_filename)

                    # Save the data as a numpy array
                    np.save(npy_file_path, data)
                    print(f"Saved {filename} as numpy array with the new filename: {new_filename}.")

if __name__ == "__main__":
    source_directory = '/data/ayc9699/dataset/pancreas_ipmn_ilkin/cyst_segmentation/labels/reoriented/cropped'
    target_directory = '/data/ayc9699/dataset/pancreas_ipmn_ilkin/cyst_segmentation/labels/reoriented/cropped/numpy'  # Replace with the actual path where you want to save the numpy files.
    #dont forget images
    convert_nii_to_numpy(source_directory, target_directory)

Saved ahn_21.nii.gz as numpy array with the new filename: ahn_21.npy.
Saved ahn_29.nii.gz as numpy array with the new filename: ahn_29.npy.
Saved ahn_54.nii.gz as numpy array with the new filename: ahn_54.npy.
Saved ahn_02.nii.gz as numpy array with the new filename: ahn_2.npy.
Saved ahn_23.nii.gz as numpy array with the new filename: ahn_23.npy.
Saved ahn_80.nii.gz as numpy array with the new filename: ahn_80.npy.
Saved ahn_51.nii.gz as numpy array with the new filename: ahn_51.npy.
Saved ahn_09.nii.gz as numpy array with the new filename: ahn_9.npy.
Saved ahn_77.nii.gz as numpy array with the new filename: ahn_77.npy.
Saved ahn_11.nii.gz as numpy array with the new filename: ahn_11.npy.


Saved ahn_48.nii.gz as numpy array with the new filename: ahn_48.npy.
Saved ahn_27.nii.gz as numpy array with the new filename: ahn_27.npy.
Saved ahn_28.nii.gz as numpy array with the new filename: ahn_28.npy.
Saved ahn_35.nii.gz as numpy array with the new filename: ahn_35.npy.
Saved ahn_16.nii.gz as numpy array with the new filename: ahn_16.npy.
Saved ahn_12.nii.gz as numpy array with the new filename: ahn_12.npy.
Saved ahn_05.nii.gz as numpy array with the new filename: ahn_5.npy.
Saved ahn_19.nii.gz as numpy array with the new filename: ahn_19.npy.
Saved ahn_41.nii.gz as numpy array with the new filename: ahn_41.npy.
Saved ahn_58.nii.gz as numpy array with the new filename: ahn_58.npy.
Saved ahn_61.nii.gz as numpy array with the new filename: ahn_61.npy.
Saved nyu_0149.nii.gz as numpy array with the new filename: nyu_149.npy.
Saved nyu_0094.nii.gz as numpy array with the new filename: nyu_94.npy.
Saved nyu_0013.nii.gz as numpy array with the new filename: nyu_13.npy.
Saved nyu_0089